In [ ]:
# This notebook performs telescope motion tests in elevation and looks at how the mirror position changes as per the measurements by the PMD

In [1]:
import sys
import asyncio
import time
import os

import numpy as np

from lsst.ts import salobj

from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS


from lsst.ts.observatory.control.utils import RotType
import logging

In [2]:
from astropy.time import Time, TimeDelta

In [3]:
# for tab completion to work
%config IPCompleter.use_jedi = False

In [4]:
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [19]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])

file:///home/patrickingraham/develop/ts_ddsconfig/config/ospl-shmem.xml
summit


In [6]:
domain = salobj.Domain()

In [8]:
atcs = ATCS(domain)
# latiss = LATISS(domain)
# await asyncio.gather(atcs.start_task, latiss.start_task)
await asyncio.gather(atcs.start_task)

atmcs: Adding all resources.
atptg: Adding all resources.
ataos: Adding all resources.
atpneumatics: Adding all resources.
athexapod: Adding all resources.
atdome: Adding all resources.
atdometrajectory: Adding all resources.
Read historical data in 0.01 sec
Read 2 history items for RemoteEvent(ATPneumatics, 0, appliedSettingsMatchStart)
Read 2 history items for RemoteEvent(ATPneumatics, 0, cellVentsState)
Read 3 history items for RemoteEvent(ATPneumatics, 0, eStop)
Read 100 history items for RemoteEvent(ATPneumatics, 0, heartbeat)
Read 3 history items for RemoteEvent(ATPneumatics, 0, instrumentState)
Read 13 history items for RemoteEvent(ATPneumatics, 0, m1CoverLimitSwitches)
Read 6 history items for RemoteEvent(ATPneumatics, 0, m1CoverState)
Read 100 history items for RemoteEvent(ATPneumatics, 0, m1SetPressure)
Read 3 history items for RemoteEvent(ATPneumatics, 0, m1State)
Read 1 history items for RemoteEvent(ATPneumatics, 0, m1VentsLimitSwitches)
Read 2 history items for RemoteEvent

[[None, None, None, None, None, None, None]]

mountPositions DDS read queue is filling: 13 of 100 elements


In [ ]:
await atcs.enable()

In [ ]:
atcs.check.atdome=False
atcs.check.atdometrajectory=False

In [ ]:
tmp=await salobj.set_summary_state(atcs.rem.atdome, salobj.State.STANDBY, settingsToApply='', timeout=30)
print(tmp)

In [ ]:
tmp=await salobj.set_summary_state(atcs.rem.atdometrajectory, salobj.State.STANDBY, settingsToApply='', timeout=30)
print(tmp)

### bring the PMDs online

In [9]:
pmd_remote = salobj.Remote(name="PMD", domain=domain, index=1)
await pmd_remote.start_task

Read historical data in 0.00 sec
Read 5 history items for RemoteEvent(PMD, 1, appliedSettingsMatchStart)
Read 100 history items for RemoteEvent(PMD, 1, heartbeat)
Read 2 history items for RemoteEvent(PMD, 1, logLevel)
Read 100 history items for RemoteEvent(PMD, 1, logMessage)
Read 20 history items for RemoteEvent(PMD, 1, summaryState)


In [15]:
tmp = await pmd_remote.evt_heartbeat.next(flush=True,timeout=15)
print(tmp)

PMDID: 1, private_revCode: f6854b27, private_sndStamp: 1627416640.945797, private_rcvStamp: 1627416640.946775, private_seqNum: 1625973, private_identity: PMD:1, private_origin: 7310, private_host: 0, heartbeat: False, priority: 0


In [ ]:
tmp = await pmd_remote.cmd_setLogLevel.set_start(level=10)

In [16]:
tmp = await pmd_remote.evt_summaryState.aget(timeout=15)
print(salobj.State(tmp.summaryState))

State.FAULT


In [22]:
tmp=await salobj.set_summary_state(pmd_remote, salobj.State.ENABLED, settingsToApply='current', timeout=30)
print(tmp)

[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]


In [24]:
tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
print(tmp)

PMDID: 1, private_revCode: 819e90fa, private_sndStamp: 1627438227.289393, private_rcvStamp: 1627438227.2902706, private_seqNum: 330789, private_identity: PMD:1, private_origin: 7310, private_host: 0, position: [-1.138, 0.475, -21.717, 11.861, 5.048, nan, nan, nan]


### Setup the script to write log messages
#### note that the index cannot be used anywhere else!

In [ ]:
#start the controller to write script messages and grab the domain
c = salobj.Controller(name="Script", index=os.getuid())

In [ ]:
c.log.setLevel(logging.DEBUG)

In [ ]:
c.log.info('Patrick Testing INFO level ') # only 

In [ ]:
az=0; el=80
c.log.info(f'az,el,pos0,pos1,pos2,pos3,pos4 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[4]}')

In [ ]:
el_samples2 = np.arange(25,83,8)
el_samples1 = np.arange(83,25,-8)

In [ ]:
print(el_samples1)
print(el_samples2)

In [ ]:
el_samples=np.concatenate((el_samples1, el_samples2))

In [ ]:
el_samples

In [ ]:
atcs.check.atdome=False
atcs.check.atdometrajectory=False

In [ ]:
print(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[4]}')

In [ ]:
az=0
c.log.info(f'Starting dip sequence for elevations of {el_samples}')
for el in el_samples:
    print(f'going to az={az}, el={el}')
    await atcs.point_azel(az=az, el=el)
    await asyncio.sleep(3)
    # grab the positions
    tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
    c.log.info(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[4]}')
    print(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[4]}')

c.log.info(f'Finished dip sequence for elevations of {el_samples}')

In [ ]:
c.log.info('Previous dataset had no ATAOS corrections enabled. M1 pressure high?') # only 

In [ ]:
await atcs.point_azel(az=az, el=82)

In [ ]:
tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
print(tmp)

In [ ]:
await atcs.rem.ataos.cmd_disableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)

In [ ]:
c.log.info('Corrections off, air valves are closed?') # only

In [ ]:
tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
print(tmp)

In [ ]:
az=0
c.log.info(f'Starting dip sequence for elevations of {el_samples}')
for el in el_samples:
    print(f'going to az={az}, el={el}')
    await atcs.point_azel(az=az, el=el)
    await asyncio.sleep(3)
    # grab the positions
    tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
    c.log.info(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[4]}')
    print(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[4]}')

c.log.info(f'Finished dip sequence for elevations of {el_samples}')

In [ ]:
tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
print(tmp)

In [ ]:
c.log.info('Corrections off, air valves are closed. Run 2') # only

In [ ]:
az=0
c.log.info(f'Starting dip sequence for elevations of {el_samples}')
for el in el_samples:
    print(f'going to az={az}, el={el}')
    await atcs.point_azel(az=az, el=el)
    await asyncio.sleep(3)
    # grab the positions
    tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
    c.log.info(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[4]}')
    print(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[4]}')

c.log.info(f'Finished dip sequence for elevations of {el_samples}')

In [ ]:
c.log.info('Corrections off, air valves are closed. Run 3') # only

In [ ]:
az=0
c.log.info(f'Starting dip sequence for elevations of {el_samples}')
for el in el_samples:
    print(f'going to az={az}, el={el}')
    await atcs.point_azel(az=az, el=el)
    await asyncio.sleep(3)
    # grab the positions
    tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
    c.log.info(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[4]}')
    print(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[4]}')

c.log.info(f'Finished dip sequence for elevations of {el_samples}')

In [ ]:
await atcs.rem.atpneumatics.cmd_openMasterAirSupply.start()

In [ ]:
await atcs.rem.atpneumatics.cmd_openInstrumentAirValve.start()

In [ ]:
await atcs.rem.atpneumatics.cmd_m1OpenAirValve.start()

In [ ]:
await atcs.rem.atpneumatics.cmd_m1SetPressure.set_start(pressure=0)

In [ ]:
await atcs.rem.ataos.cmd_disableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)

In [ ]:
c.log.info('Corrections off, air valves are open, pressure set to 0. Run 1') # only

In [ ]:
az=0
c.log.info(f'Starting dip sequence for elevations of {el_samples}')
for el in el_samples:
    print(f'going to az={az}, el={el}')
    await atcs.point_azel(az=az, el=el)
    await asyncio.sleep(3)
    # grab the positions
    tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
    c.log.info(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[4]}')
    print(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[4]}')

c.log.info(f'Finished dip sequence for elevations of {el_samples}')

In [ ]:
c.log.info('Previous dip was Corrections off, air valves are open, pressure set to 0. Run 1') # only

In [ ]:
c.log.info('Corrections off, air valves are open, pressure set to 0. Run 2') # only

In [ ]:
az=0
c.log.info(f'Starting dip sequence for elevations of {el_samples}')
for el in el_samples:
    print(f'going to az={az}, el={el}')
    await atcs.point_azel(az=az, el=el)
    await asyncio.sleep(3)
    # grab the positions
    tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
    c.log.info(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[4]}')
    print(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[4]}')

c.log.info(f'Finished dip sequence for elevations of {el_samples}')

In [ ]:
await atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True)  # m1 was at 116561 Pa

In [ ]:
await atcs.rem.ataos.cmd_resetOffset.set_start(axis='m1')

In [ ]:
await atcs.rem.ataos.cmd_resetOffset.set_start(axis='x')
await atcs.rem.ataos.cmd_resetOffset.set_start(axis='y')
await atcs.rem.ataos.cmd_resetOffset.set_start(axis='z')

In [ ]:
pressure = await atcs.rem.atpneumatics.tel_m1AirPressure.next(flush=True, timeout=10)
print(pressure)
starting_pressure = pressure.pressure

In [ ]:
scale_factor=1
await atcs.rem.ataos.cmd_resetOffset.set_start(axis='m1')
await asyncio.sleep(2)
await atcs.rem.ataos.cmd_offset.set_start(m1=starting_pressure*(1-scale_factor))
await asyncio.sleep(2)
print(pressure)
ref_pressure = await atcs.rem.atpneumatics.tel_m1AirPressure.next(flush=True, timeout=10)
c.log.info(f'Corrections on, air valves are open, pressure scale_factor set to {scale_factor}, ref_pressure now set to {ref_pressure.pressure}. Run 1') # only


In [ ]:
az=0
c.log.info(f'Starting dip sequence for elevations of {el_samples}')
for el in el_samples:
    print(f'going to az={az}, el={el}')
    await atcs.point_azel(az=az, el=el)
    await asyncio.sleep(3)
    # grab the positions
    tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
    c.log.info(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[4]}')
    print(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[4]}')

c.log.info(f'Finished dip sequence for elevations of {el_samples}')

In [ ]:
c.log.info(f'Redo test 1 since sensor 5 dropped out. Corrections on, air valves are open, pressure scale_factor set to {scale_factor}, ref_pressure now set to {ref_pressure.pressure}. Run 1') # only

In [ ]:
az=0
print(f'start time is {(Time.now()).isot}')
c.log.info(f'Starting dip sequence for elevations of {el_samples}')
for el in el_samples:
    print(f'going to az={az}, el={el}')
    await atcs.point_azel(az=az, el=el)
    await asyncio.sleep(3)
    # grab the positions
    tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
    c.log.info(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[4]}')
    print(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[4]}')
    
c.log.info(f'Finished dip sequence for elevations of {el_samples}')
print(f'End time is {(Time.now()).isot}')

In [ ]:
await atcs.point_azel(az=az, el=82)

In [ ]:
# motion now limited to stay about 25 deg elevation to avoid crazy extrapolation
# also had to reset mitutoyos

In [ ]:
offset_percentage=0.1
await atcs.rem.ataos.cmd_resetOffset.set_start(axis='m1')
await asyncio.sleep(2)
await atcs.rem.ataos.cmd_offset.set_start(m1=-(offset_percentage*starting_pressure))
await asyncio.sleep(2)
print(pressure)
ref_pressure = await atcs.rem.atpneumatics.tel_m1AirPressure.next(flush=True, timeout=10)
c.log.info(f'Corrections on, air valves are open, pressure scale_factor set to {scale_factor}, ref_pressure now set to {ref_pressure.pressure}. Run 2c') # only

In [ ]:
az=0
print(f'start time is {(Time.now()).isot}')
c.log.info(f'Starting dip sequence for elevations of {el_samples}')
for el in el_samples:
    print(f'going to az={az}, el={el}')
    await atcs.point_azel(az=az, el=el)
    await asyncio.sleep(3)
    # grab the positions
    tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
    c.log.info(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[4]}')
    print(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[4]}')
    
c.log.info(f'Finished dip sequence for elevations of {el_samples}')
print('slewing back to first position')
await atcs.point_azel(az=az, el=el_samples[0])
print(f'End time is {(Time.now()).isot}')

In [ ]:
offset_percentage=0.05
await atcs.rem.ataos.cmd_resetOffset.set_start(axis='m1')
await asyncio.sleep(2)
await atcs.rem.ataos.cmd_offset.set_start(m1=-(offset_percentage*starting_pressure))
await asyncio.sleep(2)
print(pressure)
ref_pressure = await atcs.rem.atpneumatics.tel_m1AirPressure.next(flush=True, timeout=10)
c.log.info(f'Corrections on, air valves are open, pressure scale_factor set to {scale_factor}, ref_pressure now set to {ref_pressure.pressure}. Run 1') # only

In [ ]:
az=0
print(f'start time is {(Time.now()).isot}')
c.log.info(f'Starting dip sequence for elevations of {el_samples}')
for el in el_samples:
    print(f'going to az={az}, el={el}')
    await atcs.point_azel(az=az, el=el)
    await asyncio.sleep(3)
    # grab the positions
    tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
    c.log.info(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[4]}')
    print(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[4]}')
    
c.log.info(f'Finished dip sequence for elevations of {el_samples}')
print('slewing back to first position')
await atcs.point_azel(az=az, el=el_samples[0])
print(f'End time is {(Time.now()).isot}')

In [ ]:
tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
print(tmp)

In [ ]:
await atcs.rem.atpneumatics.cmd_m1SetPressure.set_start(pressure=135000)

In [ ]:
tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
print(tmp)

In [ ]:
await atcs.rem.atpneumatics.cmd_m1SetPressure.set_start(pressure=135000)

In [ ]:
tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
print(tmp)

In [ ]:
await atcs.rem.atpneumatics.cmd_m1SetPressure.set_start(pressure=140000)

In [ ]:
tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
print(tmp)

In [ ]:
await atcs.rem.atpneumatics.cmd_m1SetPressure.set_start(pressure=0)
await atcs.point_azel(az=az, el=50)

In [ ]:
await atcs.rem.atpneumatics.cmd_m1SetPressure.set_start(pressure=0)
await asyncio.sleep(4)
tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
print(tmp)

In [ ]:
# now craig will push on it

In [ ]:
tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
print(tmp)

In [ ]:
await atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True)

In [ ]:
tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
print(tmp)

In [ ]:
import datetime
datetime.

In [ ]:
az=0
c.log.info(f'Starting dip sequence for elevations of {el_samples}')
for el in el_samples:
    print(f'going to az={az}, el={el}')
    await atcs.point_azel(az=az, el=el)
    await asyncio.sleep(3)
    # grab the positions
    tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
    c.log.info(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[4]}')
    print(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}, {tmp.position[3]}, {tmp.position[4]}')

c.log.info(f'Finished dip sequence for elevations of {el_samples}')

In [ ]:
c.log.info('Previous dataset was after craig tweaked the gauge') # only 

In [ ]:
c.log.info('Previous dataset had no active optics system enabled') # only 


In [ ]:
await atcs.point_azel(az=az, el=80)

In [ ]:
await atcs.stop_tracking()

In [ ]:
await atcs.rem.ataos.cmd_enable.start()

In [ ]:
await atcs.rem.atpneumatics.cmd_m1CloseAirValve.start()

In [ ]:
await atcs.rem.atpneumatics.cmd_closeMasterAirSupply.start()

In [ ]:
await atcs.rem.atpneumatics.cmd_closeInstrumentAirValve.start()

In [ ]:
await atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True)

In [ ]:
az=0
c.log.info('Starting run with m1 correction enabled')
for el in el_samples:
    print(f'going to az={az}, el={el}')
    await atcs.point_azel(az=az, el=el)
    # grab the positions
    tmp = await pmd_remote.tel_position.next(flush=True, timeout=25)
    c.log.info(f'az,el,pos0,pos1,pos2 = {az}, {el}, {tmp.position[0]}, {tmp.position[1]}, {tmp.position[2]}')
    

In [ ]:
await atcs.point_azel(az=0, el=80)

In [ ]:
await atcs.stop_tracking()

In [ ]:
atcs.check.atdome=True
atcs.check.atdometrajectory=True
await atcs.standby()

In [ ]:
tmp=await salobj.set_summary_state(pmd_remote, salobj.State.OFFLINE, settingsToApply='micrometer', timeout=30)
print(tmp)